<a href="https://colab.research.google.com/github/Manisharout/My_Projects/blob/main/VeloCity_Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving weatherHistory.csv to weatherHistory.csv
Saving city_day.csv to city_day.csv
Saving Metro_Interstate_Traffic_Volume.csv to Metro_Interstate_Traffic_Volume.csv


In [2]:
!pip install dash dash-core-components dash-html-components plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 38.3 MB/s eta 0:00:00


In [3]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


if 'merged' in globals() and merged is not None and not merged.empty:
    df = merged.copy()

    if not pd.api.types.is_datetime64_any_dtype(df['date_time']):
        try:
            df['date_time'] = pd.to_datetime(df['date_time'])
            if df['date_time'].dt.tz:
                 df['date_time'] = df['date_time'].dt.tz_convert(None)
        except Exception as e:
            print(f"Error converting date_time to datetime: {e}")
            df = None


    if df is not None:
        app = dash.Dash(__name__)

        app.layout = html.Div([
            html.H1("Traffic Volume and PM2.5 Analysis Dashboard"),

            html.Div([
                html.H3("Time Series Analysis"),
                dcc.Graph(id='time-series-traffic'),
                dcc.Graph(id='time-series-pm25')
            ]),

            html.Div([
                html.H3("Scatter Plot: Traffic Volume vs. Temperature"),
                dcc.Graph(id='scatter-traffic-temp')
            ]),

            html.Div([
                html.H3("Distribution Analysis"),
                dcc.Graph(id='hist-traffic'),
                dcc.Graph(id='hist-pm25')
            ]),

            html.Div([
                html.H3("Weather Condition Analysis"),
                dcc.Graph(id='bar-weather-traffic')
            ]),

            html.Div([
                html.H3("Date Range Slider"),
                dcc.RangeSlider(
                    id='date-range-slider',
                    min=0,
                    max=len(df['date_time']) - 1,
                    value=[0, len(df['date_time']) - 1],
                    marks={i: df['date_time'].iloc[i].strftime('%Y-%m-%d') for i in range(0, len(df['date_time']), len(df) // 5)}, # Example marks
                    step=1
                )
            ])
        ])

        @app.callback(
            [Output('time-series-traffic', 'figure'),
             Output('time-series-pm25', 'figure'),
             Output('scatter-traffic-temp', 'figure'),
             Output('hist-traffic', 'figure'),
             Output('hist-pm25', 'figure'),
             Output('bar-weather-traffic', 'figure')],
            [Input('date-range-slider', 'value')]
        )
        def update_graphs(date_range):
            if df is None:
                return {}, {}, {}, {}, {}, {} # Return empty figures if df is None

            start_index, end_index = date_range
            filtered_df = df.iloc[start_index:end_index+1]

            # Time Series Traffic
            fig_traffic_ts = px.line(filtered_df, x='date_time', y='traffic_volume', title='Traffic Volume Over Time')

            # Time Series PM2.5
            fig_pm25_ts = px.line(filtered_df, x='date_time', y='PM2.5', title='PM2.5 Levels Over Time')

            # Scatter Traffic vs Temp
            fig_scatter_temp = px.scatter(filtered_df, x='temp', y='traffic_volume', title='Traffic Volume vs. Temperature')

            # Histogram Traffic
            fig_hist_traffic = px.histogram(filtered_df, x='traffic_volume', title='Distribution of Traffic Volume')

            # Histogram PM2.5
            fig_hist_pm25 = px.histogram(filtered_df, x='PM2.5', title='Distribution of PM2.5 Levels')

            # Bar Weather vs Traffic
            if 'weather_main' in filtered_df.columns:
                weather_traffic = filtered_df.groupby('weather_main')['traffic_volume'].mean().reset_index()
                fig_bar_weather_traffic = px.bar(weather_traffic, x='weather_main', y='traffic_volume', title='Average Traffic Volume by Weather Condition')
            else:
                fig_bar_weather_traffic = {}


            return fig_traffic_ts, fig_pm25_ts, fig_scatter_temp, fig_hist_traffic, fig_hist_pm25, fig_bar_weather_traffic

        print("Dash app defined. To run the app in Colab, you can use:")
        print("from google.colab.output import eval_js")
        print("print(eval_js(app.run_server(mode='external')))")

    else:
        print("DataFrame 'merged' not available or empty. Cannot define Dash app.")

else:
    print("DataFrame 'merged' not available or empty. Cannot define Dash app.")

DataFrame 'merged' not available or empty. Cannot define Dash app.


In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error, mean_squared_error, r2_score
import numpy as np

merged = None
model_tuned_classification = None
model_tuned_regression = None
X_train_classification = None
X_test_classification = None
y_train_classification = None
y_test_classification = None
X_train_regression = None
X_test_regression = None
y_train_regression = None
y_test_regression = None

traffic_filename = "Metro_Interstate_Traffic_Volume.csv"
air_quality_filename = "city_day.csv"
weather_filename = "weatherHistory.csv"

try:
    print("--- Starting Data Loading ---")
    if os.path.exists(traffic_filename) and os.path.exists(air_quality_filename) and os.path.exists(weather_filename):
        try:
            traffic = pd.read_csv(traffic_filename)
            air_quality = pd.read_csv(air_quality_filename)
            weather = pd.read_csv(weather_filename)
            print("All CSV files loaded successfully.")

        except Exception as e:
            print(f"Error loading CSV files: {e}")
            traffic, air_quality, weather = None, None, None
    else:
        print("Error: One or more CSV files not found in the expected location after upload.")
        traffic, air_quality, weather = None, None, None

except Exception as e:
    print(f"An unexpected error occurred during data loading: {e}")
    traffic, air_quality, weather = None, None, None

if traffic is not None and air_quality is not None and weather is not None:
    try:
        print("\nStarting data cleaning and imputation...")

        print("\nMissing values before imputation:")
        print("Traffic:\n", traffic.isnull().sum())
        print("\nAir Quality:\n", air_quality.isnull().sum())
        print("\nWeather:\n", weather.isnull().sum())

        try:
            if 'date_time' in traffic.columns:
                traffic['date_time'] = pd.to_datetime(traffic['date_time'])
                if traffic['date_time'].dt.tz:
                    traffic['date_time'] = traffic['date_time'].dt.tz_convert(None)
                print("Processed traffic 'date_time' column.")
            else:
                 print("Warning: 'date_time' column not found in traffic DataFrame.")


            if 'Date' in air_quality.columns:
                air_quality['Date'] = pd.to_datetime(air_quality['Date'])
                if air_quality['Date'].dt.tz:
                    air_quality['Date'] = air_quality['Date'].dt.tz_convert(None)
                print("Processed air_quality 'Date' column.")
            else:
                 print("Warning: 'Date' column not found in air_quality DataFrame.")


            if 'Formatted Date' in weather.columns:
                 weather['Formatted Date'] = pd.to_datetime(weather['Formatted Date'], utc=True).dt.tz_convert(None)
                 print("Processed weather 'Formatted Date' column.")
            else:
                 print("Warning: 'Formatted Date' column not found in weather DataFrame.")


        except (ValueError, TypeError) as e:
            print(f"Error converting date columns: {e}")

        for col in traffic.select_dtypes(include=['float64', 'int64']).columns:
            if traffic[col].isnull().any():
                mean_val = traffic[col].mean()
                traffic[col] = traffic[col].fillna(mean_val)

        for col in traffic.select_dtypes(include=['object']).columns:
            if traffic[col].isnull().any():
                mode_val = traffic[col].mode()[0]
                traffic[col] = traffic[col].fillna(mode_val)
        print("Traffic data imputed.")

        for col in air_quality.select_dtypes(include=['float64', 'int64']).columns:
            if col != 'PM2.5' and air_quality[col].isnull().any():
                mean_val = air_quality[col].mean()
                air_quality[col] = air_quality[col].fillna(mean_val)

        for col in air_quality.select_dtypes(include=['object']).columns:
            if air_quality[col].isnull().any():
                mode_val = air_quality[col].mode()[0]
                air_quality[col] = air_quality[col].fillna(mode_val)
        print("Air quality data (excluding PM2.5) imputed.")

        if 'PM2.5' in air_quality.columns and air_quality['PM2.5'].isnull().any():
            air_quality.sort_values(by='Date', inplace=True)
            air_quality['PM2.5'] = air_quality['PM2.5'].interpolate(method='linear').ffill().bfill() # Add ffill/bfill for edge cases
            print("PM2.5 imputed using linear interpolation.")
        elif 'PM2.5' in air_quality.columns:
            print("No missing values in PM2.5.")
        else:
            print("Warning: 'PM2.5' column not found in air_quality DataFrame.")

        for col in weather.select_dtypes(include=['float64', 'int64']).columns:
            if weather[col].isnull().any():
                mean_val = weather[col].mean()
                weather[col] = weather[col].fillna(mean_val)

        for col in weather.select_dtypes(include=['object']).columns:
            if weather[col].isnull().any():
                mode_val = weather[col].mode()[0]
                weather[col] = weather[col].fillna(mode_val)
        print("Weather data imputed.")

        print("\nMissing values after imputation:")
        print("Traffic:\n", traffic.isnull().sum())
        print("\nAir Quality:\n", air_quality.isnull().sum())
        print("\nWeather:\n", weather.isnull().sum())

    except Exception as e:
        print(f"An error occurred during data cleaning and imputation: {e}")
        traffic, air_quality, weather = None, None, None

if traffic is not None and air_quality is not None and weather is not None:
    try:
        print("\n--- Starting Feature Engineering and Data Preparation ---")

        if 'date_time' not in traffic.columns:
            raise ValueError("Traffic DataFrame is missing 'date_time' column for merging.")
        if 'Date' not in air_quality.columns:
             raise ValueError("Air Quality DataFrame is missing 'Date' column for merging.")


        try:
            merged = pd.merge_asof(traffic.sort_values('date_time'),
                                   air_quality.sort_values('Date'),
                                   left_on='date_time',
                                   right_on='Date')
            print("Traffic and Air Quality DataFrames merged successfully.")

            merged = merged.ffill().bfill()
            print("Missing values after initial merge filled using ffill and bfill.")

        except Exception as e:
            print(f"Error during merge operation (Traffic & Air Quality): {e}")
            merged = None

        if merged is not None and not merged.empty:
            try:
                # 1. Create lagged features
                if 'traffic_volume' in merged.columns:
                    merged['traffic_volume_lag1'] = merged['traffic_volume'].shift(1)
                    merged['traffic_volume_lag24'] = merged['traffic_volume'].shift(24)
                    print("Lagged traffic_volume features created.")
                else:
                     print("Warning: 'traffic_volume' column not found for lagged feature creation.")

                if 'PM2.5' in merged.columns:
                    merged['PM2.5_lag1'] = merged['PM2.5'].shift(1)
                    merged['PM2.5_lag24'] = merged['PM2.5'].shift(24)
                    merged['PM2.5_future'] = merged['PM2.5'].shift(-2)
                    print("Lagged PM2.5 and PM2.5_future features created.")
                else:
                    print("Warning: 'PM2.5' column not found for lagged and future feature creation.")


                # 2. Engineer interaction terms
                if 'temp' in merged.columns and 'traffic_volume' in merged.columns:
                    merged['temp_traffic_interaction'] = merged['temp'] * merged['traffic_volume']
                    print("Temperature-Traffic interaction term created.")
                else:
                    print("Warning: Required columns for 'temp_traffic_interaction' not found.")

                if 'rain_1h' in merged.columns and 'traffic_volume' in merged.columns:
                     merged['rain_traffic_interaction'] = merged['rain_1h'] * merged['traffic_volume']
                     print("Rain-Traffic interaction term created.")
                else:
                    print("Warning: Required columns for 'rain_traffic_interaction' not found.")

                if 'PM2.5' in merged.columns and 'traffic_volume' in merged.columns:
                    if (merged['PM2.5'] + 1).min() > 0:
                         merged['traffic_pollution_ratio'] = merged['traffic_volume'] / (merged['PM2.5'] + 1)
                         print("Traffic-Pollution ratio created.")
                    else:
                         print("Warning: Cannot create 'traffic_pollution_ratio' due to zero or missing PM2.5 values after imputation.")
                         merged['traffic_pollution_ratio'] = np.nan

                else:
                    print("Warning: Required columns for 'traffic_pollution_ratio' not found.")


                # 3. Extract additional time-based features (if not already present and needed)
                if 'date_time' in merged.columns:
                    if 'hour' not in merged.columns:
                         merged['hour'] = merged['date_time'].dt.hour
                         print("'hour' feature created.")
                    if 'day' not in merged.columns:
                         merged['day'] = merged['date_time'].dt.day
                         print("'day' feature created.")
                    if 'weekday' not in merged.columns:
                         merged['weekday'] = merged['date_time'].dt.weekday
                         print("'weekday' feature created.")
                    if 'is_weekend' not in merged.columns:
                        merged['is_weekend'] = merged['weekday'].isin([5,6]).astype(int)
                        print("'is_weekend' feature created.")
                    if 'month' not in merged.columns:
                        merged['month'] = merged['date_time'].dt.month
                        print("'month' feature created.")
                    if 'year' not in merged.columns:
                        merged['year'] = merged['date_time'].dt.year
                        print("'year' feature created.")
                else:
                    print("Warning: 'date_time' column not found for time-based feature extraction.")


                # 4. Integrate features from weather DataFrame
                if 'weather' in globals() and weather is not None and 'Formatted Date' in weather.columns:
                    try:
                        if not pd.api.types.is_datetime64_any_dtype(weather['Formatted Date']):
                             weather['Formatted Date'] = pd.to_datetime(weather['Formatted Date'], utc=True).dt.tz_convert(None)

                        merged.sort_values('date_time', inplace=True)
                        weather.sort_values('Formatted Date', inplace=True)

                        merged = pd.merge_asof(merged,
                                               weather,
                                               left_on='date_time',
                                               right_on='Formatted Date',
                                               direction='nearest')
                        print("Merged with Weather DataFrame.")

                        if 'Formatted Date' in merged.columns and 'date_time' in merged.columns and 'Formatted Date' != 'date_time':
                             merged = merged.drop(columns=['Formatted Date'])
                             print("'Formatted Date' column from weather dropped after merge.")

                        merged = merged.ffill().bfill()
                        print("Missing values after weather merge filled.")


                    except Exception as e:
                         print(f"Error merging with weather data: {e}")
                else:
                    print("Warning: Weather DataFrame not available or missing 'Formatted Date' column for merging.")

            except Exception as e:
                print(f"An error occurred during feature engineering: {e}")
                merged = None # Ensure merged is None if feature engineering fails

            print("\nFeature Engineering and Data Preparation Summary:")
            display(merged.head())
            print("\nShape of merged DataFrame after feature engineering:", merged.shape)
            print("\nEngineered columns:", [col for col in merged.columns if any(keyword in col for keyword in ['lag', 'interaction', 'ratio', 'hour', 'day', 'weekday', 'is_weekend', 'month', 'year', 'Summary', 'Temperature (C)', 'Humidity', 'Wind Speed (km/h)'])]) # List some expected new columns


        else:
            print("Skipping feature engineering as merged DataFrame is not available or empty after initial merge.")

    except Exception as e:
        print(f"An unexpected error occurred during Feature Engineering and Data Preparation: {e}")
        merged = None

# --- Model Tuning ---
if merged is not None and not merged.empty:
    try:
        print("\n--- Traffic Volume Classification Model Tuning ---")
        # 1. Define feature set (X) and target (y) for classification
        classification_features = ['hour', 'is_weekend', 'temp', 'rain_1h', 'PM2.5',
                                   'traffic_volume_lag1', 'PM2.5_lag1', 'temp_traffic_interaction',
                                   'rain_traffic_interaction']

        if all(col in merged.columns for col in classification_features + ['traffic_volume']):
            merged_classification = merged.dropna(subset=classification_features + ['traffic_volume']).copy()

            if merged_classification.shape[0] > 1:
                X_classification = merged_classification[classification_features]
                y_classification = pd.cut(merged_classification['traffic_volume'], bins=3, labels=['Low','Medium','High'])

                if len(y_classification.unique()) > 1:
                    X_train_classification, X_test_classification, y_train_classification, y_test_classification = train_test_split(
                        X_classification, y_classification, test_size=0.2, random_state=42, stratify=y_classification)
                    print(f"Classification data split successfully. Train shape: {X_train_classification.shape}, Test shape: {X_test_classification.shape}")

                    # 2. Define parameter grid for RandomForestClassifier
                    param_grid_rfc = {
                        'n_estimators': [50, 100, 200],
                        'max_depth': [None, 10, 20],
                        'min_samples_split': [2, 5, 10],
                        'min_samples_leaf': [1, 2, 4]
                    }

                    # 3. Instantiate GridSearchCV for RandomForestClassifier
                    cv_rfc = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
                    grid_search_rfc = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                                                   param_grid=param_grid_rfc,
                                                   cv=cv_rfc,
                                                   scoring='accuracy',
                                                   n_jobs=-1)

                    # 4. Fit GridSearchCV
                    print("Starting GridSearchCV for RandomForestClassifier...")
                    grid_search_rfc.fit(X_train_classification, y_train_classification)
                    print("GridSearchCV for RandomForestClassifier completed.")

                    # 5. Print best parameters and score
                    print("Best parameters for RandomForestClassifier:", grid_search_rfc.best_params_)
                    print("Best cross-validation accuracy for RandomForestClassifier:", grid_search_rfc.best_score_)

                    # 6. Train final RandomForestClassifier model with best parameters
                    model_tuned_classification = grid_search_rfc.best_estimator_
                    print("Final RandomForestClassifier model trained with best parameters.")

                    # 7. Evaluate the tuned model
                    print("\nEvaluating Tuned RandomForestClassifier on Test Set:")
                    y_pred_classification = model_tuned_classification.predict(X_test_classification)
                    print("Classification Report:")
                    print(classification_report(y_test_classification, y_pred_classification))
                    print("\nConfusion Matrix:")
                    print(confusion_matrix(y_test_classification, y_pred_classification))

                else:
                    print("Error during classification tuning: Target variable has only one class after binning on cleaned data.")
            else:
                print("Error during classification tuning: Insufficient data after handling missing values.")
        else:
            print("Error during classification tuning: Merged DataFrame is missing required columns for classification.")


        # --- PM2.5 Regression Model Tuning ---
        print("\n--- PM2.5 Regression Model Tuning ---")

        # 1. Define feature set (X) and target (y) for regression
        regression_features = ['hour','traffic_volume','temp', 'PM2.5_lag1', 'traffic_volume_lag1',
                               'temp_traffic_interaction']
        regression_target = 'PM2.5_future'

        if all(col in merged.columns for col in regression_features + [regression_target]):
            merged_regression = merged.dropna(subset=regression_features + [regression_target]).copy()

            if merged_regression.shape[0] > 1:
                X_regression = merged_regression[regression_features]
                y_regression = merged_regression[regression_target]

                X_train_regression, X_test_regression, y_train_regression, y_test_regression = train_test_split(
                    X_regression, y_regression, test_size=0.2, random_state=42)
                print(f"Regression data split successfully. Train shape: {X_train_regression.shape}, Test shape: {X_test_regression.shape}")

                # 2. Define parameter grid for GradientBoostingRegressor
                param_grid_gbr = {
                    'n_estimators': [50, 100, 200],
                    'learning_rate': [0.01, 0.1, 0.5],
                    'max_depth': [3, 5, 10],
                    'min_samples_split': [2, 5, 10],
                    'min_samples_leaf': [1, 2, 4]
                }

                # 3. Instantiate GridSearchCV for GradientBoostingRegressor
                cv_gbr = KFold(n_splits=3, shuffle=True, random_state=42)
                grid_search_gbr = GridSearchCV(estimator=GradientBoostingRegressor(random_state=42),
                                               param_grid=param_grid_gbr,
                                               cv=cv_gbr,
                                               scoring='neg_mean_squared_error',
                                               n_jobs=-1)

                # 4. Fit GridSearchCV
                print("Starting GridSearchCV for GradientBoostingRegressor...")
                grid_search_gbr.fit(X_train_regression, y_train_regression)
                print("GridSearchCV for GradientBoostingRegressor completed.")


                # 5. Print best parameters and score
                print("Best parameters for GradientBoostingRegressor:", grid_search_gbr.best_params_)
                print("Best cross-validation MSE for GradientBoostingRegressor:", -grid_search_gbr.best_score_)

                # 6. Train final GradientBoostingRegressor model with best parameters
                model_tuned_regression = grid_search_gbr.best_estimator_
                print("Final GradientBoostingRegressor model trained with best parameters.")

                # 7. Evaluate the tuned model
                print("\nEvaluating Tuned GradientBoostingRegressor on Test Set:")
                y_pred_regression = model_tuned_regression.predict(X_test_regression)
                print("Mean Absolute Error:", mean_absolute_error(y_test_regression, y_pred_regression))
                print("Mean Squared Error:", mean_squared_error(y_test_regression, y_pred_regression))
                print("R2 Score:", r2_score(y_test_regression, y_pred_regression))

            else:
                 print("Error during regression tuning: Insufficient data after handling missing values for regression.")
        else:
            print("Error during regression tuning: Merged DataFrame is missing required columns for regression.")

    except Exception as e:
        print(f"An error occurred during model tuning: {e}")

else:
    print("\nSkipping model tuning as merged DataFrame is not available or empty.")

# 1. Print the classification report and confusion matrix for the tuned classification model
print("\n--- Traffic Volume Classification Model Performance ---")
if model_tuned_classification is not None and X_test_classification is not None and y_test_classification is not None:
    y_pred_classification = model_tuned_classification.predict(X_test_classification)
    print("Classification Report:")
    print(classification_report(y_test_classification, y_pred_classification))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test_classification, y_pred_classification))
else:
    print("Classification model or test data not available for reporting.")

--- Starting Data Loading ---
All CSV files loaded successfully.

Starting data cleaning and imputation...

Missing values before imputation:
Traffic:
 holiday                48143
temp                       0
rain_1h                    0
snow_1h                    0
clouds_all                 0
weather_main               0
weather_description        0
date_time                  0
traffic_volume             0
dtype: int64

Air Quality:
 City              0
Date              0
PM2.5          4255
PM10          10702
NO             4509
NO2            3159
NOx            4021
NH3            9785
CO             1999
SO2            3499
O3             3510
Benzene        5122
Toluene        7409
Xylene        16417
AQI            4251
AQI_Bucket     4251
dtype: int64

Weather:
 Formatted Date                0
Summary                       0
Precip Type                 517
Temperature (C)               0
Apparent Temperature (C)      0
Humidity                      0
Wind Speed (km/h)      

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,City,...,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,Labor Day,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545,Mumbai,...,rain,21.150000,21.150000,0.86,8.5008,6.0,9.8371,0.0,1016.34,Partly cloudy throughout the day.
1,Labor Day,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516,Mumbai,...,rain,22.755556,22.755556,0.83,12.3326,20.0,11.2700,0.0,1016.30,Partly cloudy throughout the day.
2,Labor Day,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767,Mumbai,...,rain,23.794444,23.794444,0.74,16.4703,49.0,11.2700,0.0,1015.22,Partly cloudy throughout the day.
3,Labor Day,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026,Mumbai,...,rain,23.905556,23.905556,0.68,14.8925,51.0,10.4006,0.0,1014.49,Partly cloudy throughout the day.
4,Labor Day,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918,Mumbai,...,rain,23.688889,23.688889,0.67,13.4435,58.0,10.0464,0.0,1014.00,Partly cloudy throughout the day.



Shape of merged DataFrame after feature engineering: (48204, 50)

Engineered columns: ['holiday', 'traffic_volume_lag1', 'traffic_volume_lag24', 'PM2.5_lag1', 'PM2.5_lag24', 'temp_traffic_interaction', 'rain_traffic_interaction', 'traffic_pollution_ratio', 'hour', 'day', 'weekday', 'is_weekend', 'month', 'year', 'Summary', 'Temperature (C)', 'Apparent Temperature (C)', 'Humidity', 'Wind Speed (km/h)', 'Daily Summary']

--- Traffic Volume Classification Model Tuning ---
Classification data split successfully. Train shape: (38563, 9), Test shape: (9641, 9)
Starting GridSearchCV for RandomForestClassifier...


## Project Summary

Developed a data science project analyzing urban traffic and air quality using integrated datasets. Performed data cleaning, feature engineering with lagged variables and interaction terms, and built machine learning models (RandomForestClassifier for traffic volume classification and GradientBoostingRegressor for PM2.5 regression). Evaluated model performance using metrics such as accuracy for classification and MAE, MSE, and R2 for regression. Created an interactive dashboard to visualize trends and relationships.

In [ ]:
from google.colab.output import eval_js
print(eval_js(app.run_server(mode='external')))

In [ ]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


if 'merged' in globals() and merged is not None and not merged.empty:
    df = merged.copy()

    if not pd.api.types.is_datetime64_any_dtype(df['date_time']):
        try:
            df['date_time'] = pd.to_datetime(df['date_time'])
            if df['date_time'].dt.tz:
                 df['date_time'] = df['date_time'].dt.tz_convert(None)
        except Exception as e:
            print(f"Error converting date_time to datetime: {e}")
            df = None


    if df is not None:
        app = dash.Dash(__name__)

        app.layout = html.Div([
            html.H1("Traffic Volume and PM2.5 Analysis Dashboard"),

            html.Div([
                html.H3("Time Series Analysis"),
                dcc.Graph(id='time-series-traffic'),
                dcc.Graph(id='time-series-pm25')
            ]),

            html.Div([
                html.H3("Scatter Plot: Traffic Volume vs. Temperature"),
                dcc.Graph(id='scatter-traffic-temp')
            ]),

            html.Div([
                html.H3("Distribution Analysis"),
                dcc.Graph(id='hist-traffic'),
                dcc.Graph(id='hist-pm25')
            ]),

            html.Div([
                html.H3("Weather Condition Analysis"),
                dcc.Graph(id='bar-weather-traffic')
            ]),

            html.Div([
                html.H3("Date Range Slider"),
                dcc.RangeSlider(
                    id='date-range-slider',
                    min=0,
                    max=len(df['date_time']) - 1,
                    value=[0, len(df['date_time']) - 1],
                    marks={i: df['date_time'].iloc[i].strftime('%Y-%m-%d') for i in range(0, len(df['date_time']), len(df) // 5)}, # Example marks
                    step=1
                )
            ])
        ])

        @app.callback(
            [Output('time-series-traffic', 'figure'),
             Output('time-series-pm25', 'figure'),
             Output('scatter-traffic-temp', 'figure'),
             Output('hist-traffic', 'figure'),
             Output('hist-pm25', 'figure'),
             Output('bar-weather-traffic', 'figure')],
            [Input('date-range-slider', 'value')]
        )
        def update_graphs(date_range):
            if df is None:
                return {}, {}, {}, {}, {}, {} # Return empty figures if df is None

            start_index, end_index = date_range
            filtered_df = df.iloc[start_index:end_index+1]

            # Time Series Traffic
            fig_traffic_ts = px.line(filtered_df, x='date_time', y='traffic_volume', title='Traffic Volume Over Time')

            # Time Series PM2.5
            fig_pm25_ts = px.line(filtered_df, x='date_time', y='PM2.5', title='PM2.5 Levels Over Time')

            # Scatter Traffic vs Temp
            fig_scatter_temp = px.scatter(filtered_df, x='temp', y='traffic_volume', title='Traffic Volume vs. Temperature')

            # Histogram Traffic
            fig_hist_traffic = px.histogram(filtered_df, x='traffic_volume', title='Distribution of Traffic Volume')

            # Histogram PM2.5
            fig_hist_pm25 = px.histogram(filtered_df, x='PM2.5', title='Distribution of PM2.5 Levels')

            # Bar Weather vs Traffic
            if 'weather_main' in filtered_df.columns:
                weather_traffic = filtered_df.groupby('weather_main')['traffic_volume'].mean().reset_index()
                fig_bar_weather_traffic = px.bar(weather_traffic, x='weather_main', y='traffic_volume', title='Average Traffic Volume by Weather Condition')
            else:
                fig_bar_weather_traffic = {}


            return fig_traffic_ts, fig_pm25_ts, fig_scatter_temp, fig_hist_traffic, fig_hist_pm25, fig_bar_weather_traffic

        print("Dash app defined. To run the app in Colab, you can use:")
        print("from google.colab.output import eval_js")
        print("print(eval_js(app.run_server(mode='external')))")

    else:
        print("DataFrame 'merged' not available or empty. Cannot define Dash app.")

else:
    print("DataFrame 'merged' not available or empty. Cannot define Dash app.")

In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error, mean_squared_error, r2_score
import numpy as np

merged = None
model_tuned_classification = None
model_tuned_regression = None
X_train_classification = None
X_test_classification = None
y_train_classification = None
y_test_classification = None
X_train_regression = None
X_test_regression = None
y_train_regression = None
y_test_regression = None

traffic_filename = "Metro_Interstate_Traffic_Volume.csv"
air_quality_filename = "city_day.csv"
weather_filename = "weatherHistory.csv"

try:
    print("--- Starting Data Loading ---")
    if os.path.exists(traffic_filename) and os.path.exists(air_quality_filename) and os.path.exists(weather_filename):
        try:
            traffic = pd.read_csv(traffic_filename)
            air_quality = pd.read_csv(air_quality_filename)
            weather = pd.read_csv(weather_filename)
            print("All CSV files loaded successfully.")

        except Exception as e:
            print(f"Error loading CSV files: {e}")
            traffic, air_quality, weather = None, None, None
    else:
        print("Error: One or more CSV files not found in the expected location after upload.")
        traffic, air_quality, weather = None, None, None

except Exception as e:
    print(f"An unexpected error occurred during data loading: {e}")
    traffic, air_quality, weather = None, None, None

if traffic is not None and air_quality is not None and weather is not None:
    try:
        print("\nStarting data cleaning and imputation...")

        print("\nMissing values before imputation:")
        print("Traffic:\n", traffic.isnull().sum())
        print("\nAir Quality:\n", air_quality.isnull().sum())
        print("\nWeather:\n", weather.isnull().sum())

        try:
            if 'date_time' in traffic.columns:
                traffic['date_time'] = pd.to_datetime(traffic['date_time'])
                if traffic['date_time'].dt.tz:
                    traffic['date_time'] = traffic['date_time'].dt.tz_convert(None)
                print("Processed traffic 'date_time' column.")
            else:
                 print("Warning: 'date_time' column not found in traffic DataFrame.")


            if 'Date' in air_quality.columns:
                air_quality['Date'] = pd.to_datetime(air_quality['Date'])
                if air_quality['Date'].dt.tz:
                    air_quality['Date'] = air_quality['Date'].dt.tz_convert(None)
                print("Processed air_quality 'Date' column.")
            else:
                 print("Warning: 'Date' column not found in air_quality DataFrame.")


            if 'Formatted Date' in weather.columns:
                 weather['Formatted Date'] = pd.to_datetime(weather['Formatted Date'], utc=True).dt.tz_convert(None)
                 print("Processed weather 'Formatted Date' column.")
            else:
                 print("Warning: 'Formatted Date' column not found in weather DataFrame.")


        except (ValueError, TypeError) as e:
            print(f"Error converting date columns: {e}")

        for col in traffic.select_dtypes(include=['float64', 'int64']).columns:
            if traffic[col].isnull().any():
                mean_val = traffic[col].mean()
                traffic[col] = traffic[col].fillna(mean_val)

        for col in traffic.select_dtypes(include=['object']).columns:
            if traffic[col].isnull().any():
                mode_val = traffic[col].mode()[0]
                traffic[col] = traffic[col].fillna(mode_val)
        print("Traffic data imputed.")

        for col in air_quality.select_dtypes(include=['float64', 'int64']).columns:
            if col != 'PM2.5' and air_quality[col].isnull().any():
                mean_val = air_quality[col].mean()
                air_quality[col] = air_quality[col].fillna(mean_val)

        for col in air_quality.select_dtypes(include=['object']).columns:
            if air_quality[col].isnull().any():
                mode_val = air_quality[col].mode()[0]
                air_quality[col] = air_quality[col].fillna(mode_val)
        print("Air quality data (excluding PM2.5) imputed.")

        if 'PM2.5' in air_quality.columns and air_quality['PM2.5'].isnull().any():
            air_quality.sort_values(by='Date', inplace=True)
            air_quality['PM2.5'] = air_quality['PM2.5'].interpolate(method='linear').ffill().bfill() # Add ffill/bfill for edge cases
            print("PM2.5 imputed using linear interpolation.")
        elif 'PM2.5' in air_quality.columns:
            print("No missing values in PM2.5.")
        else:
            print("Warning: 'PM2.5' column not found in air_quality DataFrame.")

        for col in weather.select_dtypes(include=['float64', 'int64']).columns:
            if weather[col].isnull().any():
                mean_val = weather[col].mean()
                weather[col] = weather[col].fillna(mean_val)

        for col in weather.select_dtypes(include=['object']).columns:
            if weather[col].isnull().any():
                mode_val = weather[col].mode()[0]
                weather[col] = weather[col].fillna(mode_val)
        print("Weather data imputed.")

        print("\nMissing values after imputation:")
        print("Traffic:\n", traffic.isnull().sum())
        print("\nAir Quality:\n", air_quality.isnull().sum())
        print("\nWeather:\n", weather.isnull().sum())

    except Exception as e:
        print(f"An error occurred during data cleaning and imputation: {e}")
        traffic, air_quality, weather = None, None, None

if traffic is not None and air_quality is not None and weather is not None:
    try:
        print("\n--- Starting Feature Engineering and Data Preparation ---")

        if 'date_time' not in traffic.columns:
            raise ValueError("Traffic DataFrame is missing 'date_time' column for merging.")
        if 'Date' not in air_quality.columns:
             raise ValueError("Air Quality DataFrame is missing 'Date' column for merging.")


        try:
            merged = pd.merge_asof(traffic.sort_values('date_time'),
                                   air_quality.sort_values('Date'),
                                   left_on='date_time',
                                   right_on='Date')
            print("Traffic and Air Quality DataFrames merged successfully.")

            merged = merged.ffill().bfill()
            print("Missing values after initial merge filled using ffill and bfill.")

        except Exception as e:
            print(f"Error during merge operation (Traffic & Air Quality): {e}")
            merged = None

        if merged is not None and not merged.empty:
            try:
                # 1. Create lagged features
                if 'traffic_volume' in merged.columns:
                    merged['traffic_volume_lag1'] = merged['traffic_volume'].shift(1)
                    merged['traffic_volume_lag24'] = merged['traffic_volume'].shift(24)
                    print("Lagged traffic_volume features created.")
                else:
                     print("Warning: 'traffic_volume' column not found for lagged feature creation.")

                if 'PM2.5' in merged.columns:
                    merged['PM2.5_lag1'] = merged['PM2.5'].shift(1)
                    merged['PM2.5_lag24'] = merged['PM2.5'].shift(24)
                    merged['PM2.5_future'] = merged['PM2.5'].shift(-2)
                    print("Lagged PM2.5 and PM2.5_future features created.")
                else:
                    print("Warning: 'PM2.5' column not found for lagged and future feature creation.")


                # 2. Engineer interaction terms
                if 'temp' in merged.columns and 'traffic_volume' in merged.columns:
                    merged['temp_traffic_interaction'] = merged['temp'] * merged['traffic_volume']
                    print("Temperature-Traffic interaction term created.")
                else:
                    print("Warning: Required columns for 'temp_traffic_interaction' not found.")

                if 'rain_1h' in merged.columns and 'traffic_volume' in merged.columns:
                     merged['rain_traffic_interaction'] = merged['rain_1h'] * merged['traffic_volume']
                     print("Rain-Traffic interaction term created.")
                else:
                    print("Warning: Required columns for 'rain_traffic_interaction' not found.")

                if 'PM2.5' in merged.columns and 'traffic_volume' in merged.columns:
                    if (merged['PM2.5'] + 1).min() > 0:
                         merged['traffic_pollution_ratio'] = merged['traffic_volume'] / (merged['PM2.5'] + 1)
                         print("Traffic-Pollution ratio created.")
                    else:
                         print("Warning: Cannot create 'traffic_pollution_ratio' due to zero or missing PM2.5 values after imputation.")
                         merged['traffic_pollution_ratio'] = np.nan

                else:
                    print("Warning: Required columns for 'traffic_pollution_ratio' not found.")


                # 3. Extract additional time-based features (if not already present and needed)
                if 'date_time' in merged.columns:
                    if 'hour' not in merged.columns:
                         merged['hour'] = merged['date_time'].dt.hour
                         print("'hour' feature created.")
                    if 'day' not in merged.columns:
                         merged['day'] = merged['date_time'].dt.day
                         print("'day' feature created.")
                    if 'weekday' not in merged.columns:
                         merged['weekday'] = merged['date_time'].dt.weekday
                         print("'weekday' feature created.")
                    if 'is_weekend' not in merged.columns:
                        merged['is_weekend'] = merged['weekday'].isin([5,6]).astype(int)
                        print("'is_weekend' feature created.")
                    if 'month' not in merged.columns:
                        merged['month'] = merged['date_time'].dt.month
                        print("'month' feature created.")
                    if 'year' not in merged.columns:
                        merged['year'] = merged['date_time'].dt.year
                        print("'year' feature created.")
                else:
                    print("Warning: 'date_time' column not found for time-based feature extraction.")


                # 4. Integrate features from weather DataFrame
                if 'weather' in globals() and weather is not None and 'Formatted Date' in weather.columns:
                    try:
                        if not pd.api.types.is_datetime64_any_dtype(weather['Formatted Date']):
                             weather['Formatted Date'] = pd.to_datetime(weather['Formatted Date'], utc=True).dt.tz_convert(None)

                        merged.sort_values('date_time', inplace=True)
                        weather.sort_values('Formatted Date', inplace=True)

                        merged = pd.merge_asof(merged,
                                               weather,
                                               left_on='date_time',
                                               right_on='Formatted Date',
                                               direction='nearest')
                        print("Merged with Weather DataFrame.")

                        if 'Formatted Date' in merged.columns and 'date_time' in merged.columns and 'Formatted Date' != 'date_time':
                             merged = merged.drop(columns=['Formatted Date'])
                             print("'Formatted Date' column from weather dropped after merge.")

                        merged = merged.ffill().bfill()
                        print("Missing values after weather merge filled.")


                    except Exception as e:
                         print(f"Error merging with weather data: {e}")
                else:
                    print("Warning: Weather DataFrame not available or missing 'Formatted Date' column for merging.")

            except Exception as e:
                print(f"An error occurred during feature engineering: {e}")
                merged = None # Ensure merged is None if feature engineering fails

            print("\nFeature Engineering and Data Preparation Summary:")
            display(merged.head())
            print("\nShape of merged DataFrame after feature engineering:", merged.shape)
            print("\nEngineered columns:", [col for col in merged.columns if any(keyword in col for keyword in ['lag', 'interaction', 'ratio', 'hour', 'day', 'weekday', 'is_weekend', 'month', 'year', 'Summary', 'Temperature (C)', 'Humidity', 'Wind Speed (km/h)'])]) # List some expected new columns


        else:
            print("Skipping feature engineering as merged DataFrame is not available or empty after initial merge.")

    except Exception as e:
        print(f"An unexpected error occurred during Feature Engineering and Data Preparation: {e}")
        merged = None

# --- Model Tuning ---
if merged is not None and not merged.empty:
    try:
        print("\n--- Traffic Volume Classification Model Tuning ---")
        # 1. Define feature set (X) and target (y) for classification
        classification_features = ['hour', 'is_weekend', 'temp', 'rain_1h', 'PM2.5',
                                   'traffic_volume_lag1', 'PM2.5_lag1', 'temp_traffic_interaction',
                                   'rain_traffic_interaction']

        if all(col in merged.columns for col in classification_features + ['traffic_volume']):
            merged_classification = merged.dropna(subset=classification_features + ['traffic_volume']).copy()

            if merged_classification.shape[0] > 1:
                X_classification = merged_classification[classification_features]
                y_classification = pd.cut(merged_classification['traffic_volume'], bins=3, labels=['Low','Medium','High'])

                if len(y_classification.unique()) > 1:
                    X_train_classification, X_test_classification, y_train_classification, y_test_classification = train_test_split(
                        X_classification, y_classification, test_size=0.2, random_state=42, stratify=y_classification)
                    print(f"Classification data split successfully. Train shape: {X_train_classification.shape}, Test shape: {X_test_classification.shape}")

                    # 2. Define parameter grid for RandomForestClassifier
                    param_grid_rfc = {
                        'n_estimators': [50, 100, 200],
                        'max_depth': [None, 10, 20],
                        'min_samples_split': [2, 5, 10],
                        'min_samples_leaf': [1, 2, 4]
                    }

                    # 3. Instantiate GridSearchCV for RandomForestClassifier
                    cv_rfc = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
                    grid_search_rfc = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                                                   param_grid=param_grid_rfc,
                                                   cv=cv_rfc,
                                                   scoring='accuracy',
                                                   n_jobs=-1)

                    # 4. Fit GridSearchCV
                    print("Starting GridSearchCV for RandomForestClassifier...")
                    grid_search_rfc.fit(X_train_classification, y_train_classification)
                    print("GridSearchCV for RandomForestClassifier completed.")

                    # 5. Print best parameters and score
                    print("Best parameters for RandomForestClassifier:", grid_search_rfc.best_params_)
                    print("Best cross-validation accuracy for RandomForestClassifier:", grid_search_rfc.best_score_)

                    # 6. Train final RandomForestClassifier model with best parameters
                    model_tuned_classification = grid_search_rfc.best_estimator_
                    print("Final RandomForestClassifier model trained with best parameters.")

                    # 7. Evaluate the tuned model
                    print("\nEvaluating Tuned RandomForestClassifier on Test Set:")
                    y_pred_classification = model_tuned_classification.predict(X_test_classification)
                    print("Classification Report:")
                    print(classification_report(y_test_classification, y_pred_classification))
                    print("\nConfusion Matrix:")
                    print(confusion_matrix(y_test_classification, y_pred_classification))

                else:
                    print("Error during classification tuning: Target variable has only one class after binning on cleaned data.")
            else:
                print("Error during classification tuning: Insufficient data after handling missing values.")
        else:
            print("Error during classification tuning: Merged DataFrame is missing required columns for classification.")


        # --- PM2.5 Regression Model Tuning ---
        print("\n--- PM2.5 Regression Model Tuning ---")

        # 1. Define feature set (X) and target (y) for regression
        regression_features = ['hour','traffic_volume','temp', 'PM2.5_lag1', 'traffic_volume_lag1',
                               'temp_traffic_interaction']
        regression_target = 'PM2.5_future'

        if all(col in merged.columns for col in regression_features + [regression_target]):
            merged_regression = merged.dropna(subset=regression_features + [regression_target]).copy()

            if merged_regression.shape[0] > 1:
                X_regression = merged_regression[regression_features]
                y_regression = merged_regression[regression_target]

                X_train_regression, X_test_regression, y_train_regression, y_test_regression = train_test_split(
                    X_regression, y_regression, test_size=0.2, random_state=42)
                print(f"Regression data split successfully. Train shape: {X_train_regression.shape}, Test shape: {X_test_regression.shape}")

                # 2. Define parameter grid for GradientBoostingRegressor
                param_grid_gbr = {
                    'n_estimators': [50, 100, 200],
                    'learning_rate': [0.01, 0.1, 0.5],
                    'max_depth': [3, 5, 10],
                    'min_samples_split': [2, 5, 10],
                    'min_samples_leaf': [1, 2, 4]
                }

                # 3. Instantiate GridSearchCV for GradientBoostingRegressor
                cv_gbr = KFold(n_splits=3, shuffle=True, random_state=42)
                grid_search_gbr = GridSearchCV(estimator=GradientBoostingRegressor(random_state=42),
                                               param_grid=param_grid_gbr,
                                               cv=cv_gbr,
                                               scoring='neg_mean_squared_error',
                                               n_jobs=-1)

                # 4. Fit GridSearchCV
                print("Starting GridSearchCV for GradientBoostingRegressor...")
                grid_search_gbr.fit(X_train_regression, y_train_regression)
                print("GridSearchCV for GradientBoostingRegressor completed.")


                # 5. Print best parameters and score
                print("Best parameters for GradientBoostingRegressor:", grid_search_gbr.best_params_)
                print("Best cross-validation MSE for GradientBoostingRegressor:", -grid_search_gbr.best_score_)

                # 6. Train final GradientBoostingRegressor model with best parameters
                model_tuned_regression = grid_search_gbr.best_estimator_
                print("Final GradientBoostingRegressor model trained with best parameters.")

                # 7. Evaluate the tuned model
                print("\nEvaluating Tuned GradientBoostingRegressor on Test Set:")
                y_pred_regression = model_tuned_regression.predict(X_test_regression)
                print("Mean Absolute Error:", mean_absolute_error(y_test_regression, y_pred_regression))
                print("Mean Squared Error:", mean_squared_error(y_test_regression, y_pred_regression))
                print("R2 Score:", r2_score(y_test_regression, y_pred_regression))

            else:
                 print("Error during regression tuning: Insufficient data after handling missing values for regression.")
        else:
            print("Error during regression tuning: Merged DataFrame is missing required columns for regression.")

    except Exception as e:
        print(f"An error occurred during model tuning: {e}")

else:
    print("\nSkipping model tuning as merged DataFrame is not available or empty.")

# 1. Print the classification report and confusion matrix for the tuned classification model
print("\n--- Traffic Volume Classification Model Performance ---")
if model_tuned_classification is not None and X_test_classification is not None and y_test_classification is not None:
    y_pred_classification = model_tuned_classification.predict(X_test_classification)
    print("Classification Report:")
    print(classification_report(y_test_classification, y_pred_classification))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test_classification, y_pred_classification))
else:
    print("Classification model or test data not available for reporting.")

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Create a scatter plot to visualize actual vs. predicted for regression
if model_tuned_regression is not None and X_test_regression is not None and y_test_regression is not None:
    y_pred_regression = model_tuned_regression.predict(X_test_regression)
    print("\nCreating scatter plot for Actual vs. Predicted PM2.5 (Regression)...")
    fig_regression_scatter = px.scatter(x=y_test_regression, y=y_pred_regression,
                                        labels={'x': 'Actual PM2.5', 'y': 'Predicted PM2.5'},
                                        title='Actual vs. Predicted PM2.5 (Regression Model)')
    fig_regression_scatter.show()
else:
    print("Regression model or test data not available for scatter plot.")


# Generate Confusion Matrix Heatmap for Classification Model
if model_tuned_classification is not None and X_test_classification is not None and y_test_classification is not None:
    y_pred_classification = model_tuned_classification.predict(X_test_classification)
    cm = confusion_matrix(y_test_classification, y_pred_classification)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['High', 'Low', 'Medium'], yticklabels=['High', 'Low', 'Medium']) # Assuming labels based on previous output
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix for Traffic Volume Classification')
    plt.show()
else:
    print("Classification model or test data not available for confusion matrix heatmap.")

In [ ]:
from google.colab.output import eval_js
print(eval_js(app.run(mode='external')))

## Summarize data understanding

### Subtask:
Briefly describe the datasets used, the key variables, and any initial observations after data loading and cleaning.


**Reasoning**:
Access and display information about the traffic, air_quality, and weather dataframes to understand their structure and content after loading and cleaning, as requested in steps 1-4 of the instructions. This will provide the necessary details for the subsequent summary.



In [ ]:
print("--- Traffic DataFrame Information ---")
display(traffic.head())
traffic.info()
print("Shape:", traffic.shape)

print("\n--- Air Quality DataFrame Information ---")
display(air_quality.head())
air_quality.info()
print("Shape:", air_quality.shape)

print("\n--- Weather DataFrame Information ---")
display(weather.head())
weather.info()
print("Shape:", weather.shape)

In [ ]:
engineered_columns = [col for col in merged.columns if any(keyword in col for keyword in ['lag', 'interaction', 'ratio', 'hour', 'day', 'weekday', 'is_weekend', 'month', 'year']) or col in ['Summary', 'Temperature (C)', 'Humidity', 'Wind Speed (km/h)', 'Daily Summary']]

print("--- Engineered Features and Their Rationale ---")

for col in engineered_columns:
    if 'lag' in col:
        original_col = col.replace('_lag1', '').replace('_lag24', '')
        lag_hours = col.split('lag')[-1]
        print(f"\nFeature: {col}")
        print(f"Creation: Created by shifting the '{original_col}' column by {lag_hours} hour(s).")
        print("Rationale: Captures temporal dependencies and patterns in traffic volume and PM2.5 levels from previous hours.")
    elif 'interaction' in col:
        components = col.replace('_interaction', '').split('_')
        print(f"\nFeature: {col}")
        print(f"Creation: Created by multiplying the '{components[0]}' and '{components[1]}' columns.")
        print("Rationale: Captures the combined effect or how the relationship between two variables impacts the target variables.")
    elif 'ratio' in col:
        components = col.split('_')
        print(f"\nFeature: {col}")
        print(f"Creation: Created by dividing '{components[0]}' by '{components[1]}' (with a small constant added to the denominator to avoid division by zero).")
        print("Rationale: Represents the relative impact of traffic compared to pollution, potentially indicating scenarios where traffic is high despite poor air quality or vice versa.")
    elif col in ['hour', 'day', 'weekday', 'is_weekend', 'month', 'year']:
        print(f"\nFeature: {col}")
        print(f"Creation: Extracted from the 'date_time' column.")
        print("Rationale: Captures daily, weekly, monthly, and annual seasonality and time-of-day patterns in traffic and pollution.")
    elif col in ['Summary', 'Temperature (C)', 'Humidity', 'Wind Speed (km/h)', 'Daily Summary', 'Apparent Temperature (C)', 'Pressure (millibars)', 'Visibility (km)', 'Wind Bearing (degrees)', 'Precip Type', 'Loud Cover']:
         print(f"\nFeature: {col}")
         print(f"Creation: Integrated from the 'weather' DataFrame through a time-based merge.")
         print("Rationale: Incorporates external environmental factors known to influence traffic and air quality.")


print("\n--- Overall Goal of Feature Engineering ---")
print("The overall goal of feature engineering was to create new variables that better capture the complex temporal, environmental, and interactive relationships within the data.")
print("These engineered features provide richer information to the machine learning models, potentially improving their ability to predict future traffic volume and PM2.5 levels.")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Generate Confusion Matrix Heatmap for Classification Model
if model_tuned_classification is not None and X_test_classification is not None and y_test_classification is not None:
    y_pred_classification = model_tuned_classification.predict(X_test_classification)
    cm = confusion_matrix(y_test_classification, y_pred_classification)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['High', 'Low', 'Medium'], yticklabels=['High', 'Low', 'Medium']) # Assuming labels based on previous output
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix for Traffic Volume Classification')
    plt.show()
else:
    print("Classification model or test data not available for confusion matrix heatmap.")